This notebook provides examples of extracting different types of metadata that describes the "truth".

The notebook is good for inspecting one event. For an example of a script that would extract information 
from one or more files, refer to `script_mcinfo_example.py` in the same folder as this notebook.

In [ ]:
import ROOT as rt
from larlite import larlite
from ublarcvapp import ublarcvapp

In [ ]:
# make a list of input files you want to load
# these should be "larlite mcinfo files" which contain the following ROOT trees
#
"""
  KEY: TTree	mctruth_corsika_tree;1	mctruth Tree by corsika
  KEY: TTree	mctruth_generator_tree;1	mctruth Tree by generator
  KEY: TTree	mcflux_generator_tree;1	mcflux Tree by generator
  KEY: TTree	mcshower_mcreco_tree;1	mcshower Tree by mcreco
  KEY: TTree	mctrack_mcreco_tree;1	mctrack Tree by mcreco
"""
# you can check for these trees by opening the ROOT file with the ROOT interpreter
"""
> root -l file.root
root[1] .ls
"""

f_v = ["../../testdata/mcc9_v29e_dl_run3b_bnb_intrinsic_nue_overlay_nocrtremerge/merged_dlreco_001361e0-3306-491f-9098-1d08eee8458b.root"]

In [ ]:
# We create an instance of the larlite.storage_manager class which provides our interface to the file
io = larlite.storage_manager( larlite.storage_manager.kREAD )
for f in f_v:
    io.add_in_filename( f )
io.open()

In [ ]:
# Load an entry
io.go_to(0)

As a quick reference, here is the picture of the scattering interaction (just replace the electron with a neutrino). The slides shows the kinematic variables we track for the incoming and out-going particles.
(Note to self, get/draw a neutrino picture.)

Slide from "Particle Physics: Handout 6 Deep inelastic Scattering" by Mark Thomas.
Ref: https://www.hep.phy.cam.ac.uk/~thomson/partIIIparticles/handouts/Handout6_2010.pdf

<img src="scattering_variables.png"
     alt="Scattering Variables"
     style="float: left; margin-right: 10px; width: 100px;" />
     
<img src="scattering_variables_p2.png"
     alt="Scattering Variables, slide 2"
     style="float: left; margin-right: 10px; width: 100px;" />


Also, it might help to brielfly outline the inputs that go into simulating a beam event.

First we need to have the beam flux. This tells us what the energy distribution is for the different neutrino flavors: muon neutrinos, anti-muon neutrinos, electron neutrinos, and anti-electron neutrinos. 
We then sample from these flux distributions to get a neutrino of a certain flavor and energy.

Next, we give that info to the GENIE neutrino interaction generator. 
The generator samples a possible neutrino interaction with the argon nucleus based on
the theoretical models the collabortion has configured.
The GENIE picks things like the interaction mode, the theoretical class of interaction that occured,
and specifies the particles that emerge from the nucleus.
These output particles are known as the final state particles.

Then, the final state particles from GENIE is given to a Geant4 simulation of the detector.
Geant's algoriths transport each of the final state particles through the detector.
Along the way, a particle can do lots of things. Examples include: s
imply lose energy into the detector medium through ioninzation,
scatter with the nuclei in the detector to produce daughter particles,
or decay into daughter particles. 
Any particles produced during transport are also transported until all 
particles fall below some energy threshold.

Finally, the amount of location of energy given to the detector medium (the liquid argon)
by the particles is recorded and used to simulate the wire signals.
Distortion from the electronics and noise is added to the raw signals.
This then produces a data (which for us is image-like) that fairly closely resembles the data produced by the data.
For some more info on how you go from ionization in the detector to wire signals, see this notebook:
https://sites.tufts.edu/nutufts/projects/lartpc-data-tutorial/ 

Additionally, cosmic ray trajectories might be introduced into the event. 
This is important as the detectors in the SBN program (including MicroBooNE),
are on the surface and will have cosmic ray particles crossing it constantly.

We can either:
 * simulate cosmic rays from a generator, e.g. Corsika, and give those particles to Geant4 to transport along with those from GENIE.
 * We can wire signals from off-beam data, which should contain no neutrinos from the beam, and overlay it onto the simulated wire signals.
 
For the former (simulated cosmics), we will have truth information similar to the neutrino interaction particles. 
For the latter (off-beam data signals), we will not.


In [ ]:
# The Neutrino Interaction Information

# this is a container of larlite::gtruth instances
# https://github.com/NuTufts/larlite/blob/nutufts/larlite/DataFormat/gtruth.h
# This class contains information from the GENIE interaction generator about 
#   the sampled neutrino-nucleus interaction.
# The container below holds info on all the interactions in the event. For MicroBooNE,
#   this should hold 1 event (very improbable that 2 interactions would occur in a beam event)
ev_gtruth = io.get_data( larlite.data.kGTruth, "generator" )

# get the first (and probably only) interaction in this event
gtruth = ev_gtruth.at(0)

# we can print out some info. Reference the gtruth.h header above to get more informaiton.

print("Interaction code: ",gtruth.fGint)
print("Neutrino Scattering code: ",gtruth.fGscatter)
print("Q^2, 4-momentum transfered from the neutrino to the nucleus: ",gtruth.fgQ2)
print("W, #nu in the slides above, E3-E1: ",gtruth.fgW)
print("X, Bjorken X: ",gtruth.fgX)
print("X, Bjorken Y, fraction of neutrino energy transferred to nucleus: ",gtruth.fgY)

In [ ]:
# MCTruth object, info about incoming neutrino, interaction, and final state particles
# Note that some of the information from the gtruth object above has been copied into this object.
# Typically, you'll want to use this object to get at the most common GENIE parameters.
# gtruth gives more expert-level parameters.

# get container of larlite::mctruth instances
# https://github.com/NuTufts/larlite/blob/nutufts/larlite/DataFormat/mctruth.h
ev_mctruth = io.get_data(larlite.data.kMCTruth, "generator")

# get the first (and only for MicroBooNE) interaction in the container
mctruth = ev_mctruth.at(0)

# Get info about the neutrino involved in the interaction
# Stored in a larlite::mcnu class instance
# https://github.com/NuTufts/larlite/blob/nutufts/larlite/DataFormat/mcnu.h
mcnu = mctruth.GetNeutrino()
print("Neutrino PdgCode (flavor): ",mcnu.Nu().PdgCode())
print("Charged Current (0) or Neutral Current (1): ",mcnu.CCNC())
print("GENIE Mode: ",mcnu.Mode())
print("Interaction type (NUANCE Code): ",mcnu.InteractionType())
print("Neutrino Energy: ",mcnu.Nu().Momentum().E()," GeV")
print("Position: ",[mcnu.Nu().Position()(x) for x in range(4)])
print("  where [x cm, the drift dim; y cm, the vertical dim; z cm, the beam dim; geant4 time ns])")

# For interpretation of the GENIE Codes
#   see https://github.com/twongjirad/LArLiteSoftCookBook/wiki/Interaction-Mode-list
# For interpretation of the NUANCE Codes
#   see https://github.com/LArbys/dllee_unified/wiki/Interaction-Modes
#   how these modes are set (super weedsy): https://github.com/twongjirad/LArLiteSoftCookBook/wiki/How-is-the-mode-set


In the next few cells, we'll be dumping out information on the particles involved in the event.

The particle type is encoded as "PDG Codes" that particle experiments somehow have agreed to use.
A full list is here: https://pdg.lbl.gov/2007/reviews/montecarlorpp.pdf

A short list of the most relevent codes are here. Note that negative numbers is the anti-particle code.



| Code | Particle |
| ---- | ----- |
|  11  | electron |
|  13  | muon |
|  12  | electron neutrino |
|  14  | muon neutrino |
|  22  | photon |
|  2212 | proton |
|  2112 | neutron |
|  211  | charged pion |
|  111  | neutral pion |
|  1000010020  | (or large number) This is a nucleus |

In [ ]:
# Simulated Particle Trajectories
# The truth metadata includes the list of particles the Geant4 library simulated through our detector
# The trajectories are split into two kinds: tracks and shower

# Note that there are IDs that keep track of the different particle trajectories and their relationships
# There is the
# * trackID: a number assigned to any particle transported by Geant. Assigned by order it's created usually.
# * MotherID: the particle trajectory that created this particle
# * AncestorID: the original particle trajectory given to Geant that kicked off the whole cascade or particles

# Important set of particles are the "primaries"
# These particles were given to the simulation as input (and come from GENIE or a cosmic ray simulation for example)
# Primary particles have the same number for their track, mother, and ancestor IDs.

# Another important variable is Origin
# Origin=1 is from a neutrino generator (GENIE)
# Origin=2 is from a cosmic particle generator (Corsika)

ev_tracks = io.get_data( larlite.data.kMCTrack, "mcreco" )
for itrack in range(ev_tracks.size()):
    track = ev_tracks.at(itrack)
    # track is an instance of the larlite::mctrack class
    # https://github.com/NuTufts/larlite/blob/nutufts/larlite/DataFormat/mctrack.h
    
    # we're going to skip the neutrons: they don't make a lot of tracks we can see in the detector
    if track.PdgCode()==2112:
        continue
    data = (itrack,track.PdgCode(),track.Start().E(),track.TrackID(), 
            track.MotherTrackID(), track.AncestorTrackID(),track.Origin())
    print("[track #%d] pdgcode=%d energy=%.2f MeV geantid=%d motherid=%d ancestorid=%d origin=%d"%(data))
    
ev_showers = io.get_data( larlite.data.kMCShower, "mcreco" )
for ishower in range(ev_showers.size()):
    shower = ev_showers.at(ishower)
    # track is an instance of the larlite::mctrack class
    # https://github.com/NuTufts/larlite/blob/nutufts/larlite/DataFormat/mctrack.h
    
    # we're going to skip the neutrons: they don't make a lot of tracks we can see in the detector
    if shower.PdgCode()==2112:
        continue
    data = (ishower,shower.PdgCode(),shower.Start().E(),shower.TrackID(), 
            shower.MotherTrackID(), shower.AncestorTrackID(),shower.Origin())
    print("[shower #%d] pdgcode=%d energy=%.2f MeV geantid=%d motherid=%d ancestorid=%d origin=%d"%(data))


In [ ]:
# Particle graph

# The mctrack and mcshower containers are just a list of particles.
# It is often convenient to try and organize this information in a way
#   that captures the mother-daughter relationship between particles.

# The class ublarcvapp::mctools::MCPixelPGraph is meant to help with this.
# To see more about the class: https://github.com/LArbys/ublarcvapp/blob/dlgen2/ublarcvapp/MCTools/MCPixelPGraph.h

# create instance of algorithm
mcpg = ublarcvapp.mctools.MCPixelPGraph()

# pass the io manager (after setting the entry with go_to). 
# the function will then read mctrack and mcshower info and assemble a graph
mcpg.buildgraphonly( io )

# dump it out
# Note that the node with origin=-1 is meant to represent the interacting neutrino and provide a root node
#   for all the neutrino interaction final state particles
mcpg.printGraph(0,False)